# monthly-builder

In [1]:
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.common.by import By
# from bs4 import BeautifulSoup

# from funcs import create_game_info, create_team_info, create_info_df, create_boxscores, merge_boxscores, change_dtypes, create_PIE, commit_data

# import pandas as pd
# import numpy as np
# import string
# import time
# import sqlite3
# import warnings
# warnings.filterwarnings('ignore')

# pd.set_option('display.max_columns', None)

%run funcs.py

In [2]:
# global variables

# used to create id strings later
base_url = 'https://www.basketball-reference.com'

season_gamecount = 1

precovid_seasons = ['1314', '1415', '1516', '1617', '1718', '1819']
precovid_url_years = ['2014', '2015', '2016', '2017', '2018', '2019']
postcovid_seasons = ['1920', '2021', '2122', '2223']
postcovid_url_years = ['2020', '2021', '2022', '2023']

# post_covid_season_dict = {'1920': {'month_len': 8, 'final_month_gamecount': 83},
#                           '2021': {'month_len': 6, 'final_month_gamecount': 140},
#                           '2122': {'month_len': 7, 'final_month_gamecount': 83},
#                           '2223': {'month_len': 7, 'final_month_gamecount': 72}
#                          }

# used to create sql database table columns

info_columns = ['game_id', 'season', 'date', 'type', 'away_team', 'away_score', 'home_team', 'home_score', 'result']
num_columns = ['MP', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-',
                'FG%', '3P%', 'FT%', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']

basic_stats = ['MP', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-']
# advanced_stats = ['FG%', '3P%', 'FT%', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']

rand = 6

In [ ]:
# connect to sql database
conn = sqlite3.connect('data/temp/postcovid-current-season.sqlite')
driver = webdriver.Chrome(service=svc,options=options)
time.sleep(rand)

# season_id = '1617'
# url_year = '2017'

# precovid_seasons = ['1617', '1718', '1819']
# precovid_url_years = ['2017', '2018', '2019']
postcovid_seasons = ['2224']
postcovid_url_years = ['2024']

In [ ]:
for i in range(len(postcovid_seasons)):
    
    season_id = postcovid_seasons[i]
    url_year = postcovid_url_years[i]
    season_gamecount = 1
    game_type = 'regular'
    start_url = 'https://www.basketball-reference.com/leagues/NBA_' + url_year + '_games.html'
    
    # open the season schedule page
    driver.get(start_url)
    time.sleep(rand)
    src = driver.page_source
    # create beautiful soup object from html/xml
    parser = BeautifulSoup(src, 'lxml')
    
    # every month from the season
    months = parser.find('div', attrs = {'class': 'filter'})
    # partial urls for each month
    links = months.findAll('a')
    # full urls for each month
    month_links = [base_url + link['href'] for link in links]
    month_links =month_links[:2]
    
    # only include regular season months
    # month_links = month_links[0:post_covid_season_dict[season_id]['month_len']]

    
    for month_url in month_links:
        
        driver.get(month_url)
        time.sleep(rand)
        month_src = driver.page_source
        month_parser = BeautifulSoup(month_src, 'lxml')
        month_table = month_parser.find('div', attrs = {'class': 'table_container is_setup'})

        #check if there are play-in games this month
        play_in_str = month_table.findAll('td', string='Play-In Game')

        # if no play-in games:
        if not play_in_str:
        
            game_partial_urls = month_table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
            game_urls = [base_url + url.a['href'] for url in game_partial_urls]
        
            for i in range(len(game_urls)):
        
                # driver.quit()
                # time.sleep(rand)
                # driver = webdriver.Chrome(service=svc,options=options)
                # time.sleep(rand)
                # driver.maximize_window()
                driver.get(game_urls[i])
                time.sleep(rand)
                src = driver.page_source
                parser = BeautifulSoup(src, 'lxml')
        
                # game_info database:
        
                id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
                game_info = create_game_info(url=game_urls[i],
                                             season_id=season_id,
                                             season_gamecount=season_gamecount,
                                             game_type=game_type)
                # will use game_id with create_boxscores()
                game_id = game_info[0]
                team_info = create_team_info(id_table)
                # will use team_ids with merge_boxscores()
                team_ids = [team_info[0], team_info[2]]
        
                info_df = create_info_df(game_info=game_info,
                                         team_info=team_info,
                                         info_columns=info_columns)
                # write game info to sql database
                info_df.to_sql('game_info', con=conn, if_exists='append', index=False)
        
                # team/player databases:
        
                # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
                stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})
        
                player_box_list = [None, None, None, None]
                team_box_list = [None, None, None, None]
        
                # create team and player boxscores
                for i in range(len(stat_tables)):
                    # split player and team boxscores
                    player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)
        
                # team_stats database:
        
                # combine boxscore and advanced boxscore for each team
                away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
                team_boxes = pd.concat([away_team_box, home_team_box])
                team_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                team_boxes = change_dtypes(team_boxes, num_columns)
                # write to sql database
                team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)
        
                # player_stats database:
        
                # combine boxscore and advanced boxscore for each team
                away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
                player_boxes = pd.concat([away_player_box, home_player_box])
                player_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                player_boxes = change_dtypes(player_boxes, num_columns)
                # create team totals for PIE calculation
                totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
                # add PIE column to player boxscore
                player_boxes = create_PIE(player_boxes, totals)
                # write to sql database
                player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)
        
                
                button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')
        
                buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')
        
                del buttons[6]
                del buttons[3]
                del buttons[0]
        
                game_length = len(buttons)
        
                button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
                game_button_codes = button_codes[:game_length]
        
                for team_id in team_ids:
                    for period in game_button_codes:
        
                        id_string = f'box-{team_id}-{period}-basic'
                        period_table = parser.find('table', attrs = {'id': id_string})
        
                        player_box, team_box = create_boxscores(period_table, game_id=game_id)
        
                        # team_stats database:
        
                        team_box.rename(columns={'Starters': 'team'}, inplace=True)
                        team_box['team'] = team_id
                        team_box.insert(loc=1, column='period', value=period)
        
                        team_box.reset_index(drop=True, inplace=True)
                        team_box = change_dtypes(team_box, basic_stats)
        
                        team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)
        
                        # player_stats database:
        
                        player_box.insert(loc=2, column='team', value=team_id)
                        player_box.insert(loc=1, column='period', value=period)
        
                        player_box.reset_index(drop=True, inplace=True)
                        player_box = change_dtypes(player_box, basic_stats)
        
                        player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)
        
                # increase gamecount to create next game_id
                season_gamecount += 1
                time.sleep(rand)
        
        elif play_in_str:
        
            play_in_rows = []
            for cell in play_in_str:
                parent = cell.find_parent()
                row = int(parent['data-row'])
                play_in_rows.append(row)
            
            first_play_in = play_in_rows[0]
            last_play_in = play_in_rows[-1]
            first_playoff = last_play_in + 1
            
            parent_table = play_in_str[0].find_parent().find_parent()
            slip_headers = parent_table.findAll('tr', attrs = {'class': 'thead'})
            
            for header in slip_headers:
                header_row = int(header['data-row'])
                if header_row < first_play_in:
                    first_play_in -= 1
                if header_row < last_play_in:
                    first_playoff -= 1
            
            game_partial_urls = month_table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
            reg_game_urls = [base_url + url.a['href'] for url in game_partial_urls[:first_play_in]]
            play_in_game_urls = [base_url + url.a['href'] for url in game_partial_urls[first_play_in:first_playoff]]
            playoff_game_urls = [base_url + url.a['href'] for url in game_partial_urls[first_playoff:]]
        
        
            for i in range(len(reg_game_urls)):
        
                # driver.quit()
                # time.sleep(rand)
                # driver = webdriver.Chrome(service=svc,options=options)
                # time.sleep(rand)
                # driver.maximize_window()
                driver.get(reg_game_urls[i])
                time.sleep(rand)
                src = driver.page_source
                parser = BeautifulSoup(src, 'lxml')
        
                # game_info database:
        
                id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
                game_info = create_game_info(url=reg_game_urls[i],
                                             season_id=season_id,
                                             season_gamecount=season_gamecount,
                                             game_type=game_type)
                # will use game_id with create_boxscores()
                game_id = game_info[0]
                team_info = create_team_info(id_table)
                # will use team_ids with merge_boxscores()
                team_ids = [team_info[0], team_info[2]]
        
                info_df = create_info_df(game_info=game_info,
                                         team_info=team_info,
                                         info_columns=info_columns)
                # write game info to sql database
                info_df.to_sql('game_info', con=conn, if_exists='append', index=False)
        
                # team/player databases:
        
                # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
                stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})
        
                player_box_list = [None, None, None, None]
                team_box_list = [None, None, None, None]
        
                # create team and player boxscores
                for i in range(len(stat_tables)):
                    # split player and team boxscores
                    player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)
        
                # team_stats database:
        
                # combine boxscore and advanced boxscore for each team
                away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
                team_boxes = pd.concat([away_team_box, home_team_box])
                team_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                team_boxes = change_dtypes(team_boxes, num_columns)
                # write to sql database
                team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)
        
                # player_stats database:
        
                # combine boxscore and advanced boxscore for each team
                away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
                player_boxes = pd.concat([away_player_box, home_player_box])
                player_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                player_boxes = change_dtypes(player_boxes, num_columns)
                # create team totals for PIE calculation
                totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
                # add PIE column to player boxscore
                player_boxes = create_PIE(player_boxes, totals)
                # write to sql database
                player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)
        
                
                button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')
        
                buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')
        
                del buttons[6]
                del buttons[3]
                del buttons[0]
        
                game_length = len(buttons)
        
                button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
                game_button_codes = button_codes[:game_length]
        
                for team_id in team_ids:
                    for period in game_button_codes:
        
                        id_string = f'box-{team_id}-{period}-basic'
                        period_table = parser.find('table', attrs = {'id': id_string})
        
                        player_box, team_box = create_boxscores(period_table, game_id=game_id)
        
                        # team_stats database:
        
                        team_box.rename(columns={'Starters': 'team'}, inplace=True)
                        team_box['team'] = team_id
                        team_box.insert(loc=1, column='period', value=period)
        
                        team_box.reset_index(drop=True, inplace=True)
                        team_box = change_dtypes(team_box, basic_stats)
        
                        team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)
        
                        # player_stats database:
        
                        player_box.insert(loc=2, column='team', value=team_id)
                        player_box.insert(loc=1, column='period', value=period)
        
                        player_box.reset_index(drop=True, inplace=True)
                        player_box = change_dtypes(player_box, basic_stats)
        
                        player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)
        
                # increase gamecount to create next game_id
                season_gamecount += 1
        
            # game_type = 'play-in'
            # for i in range(len(play_in_game_urls)):
        
            #     # driver.quit()
            #     # time.sleep(rand)
            #     # driver = webdriver.Chrome(service=svc,options=options)
            #     # time.sleep(rand)
            #     # driver.maximize_window()
            #     driver.get(play_in_game_urls[i])
            #     time.sleep(rand)
            #     src = driver.page_source
            #     parser = BeautifulSoup(src, 'lxml')
        
            #     # game_info database:
        
            #     id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
            #     game_info = create_game_info(url=play_in_game_urls[i],
            #                                  season_id=season_id,
            #                                  season_gamecount=season_gamecount,
            #                                  game_type=game_type)
            #     # will use game_id with create_boxscores()
            #     game_id = game_info[0]
            #     team_info = create_team_info(id_table)
            #     # will use team_ids with merge_boxscores()
            #     team_ids = [team_info[0], team_info[2]]
        
            #     info_df = create_info_df(game_info=game_info,
            #                              team_info=team_info,
            #                              info_columns=info_columns)
            #     # write game info to sql database
            #     info_df.to_sql('game_info', con=conn, if_exists='append', index=False)
        
            #     # team/player databases:
        
            #     # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
            #     stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})
        
            #     player_box_list = [None, None, None, None]
            #     team_box_list = [None, None, None, None]
        
            #     # create team and player boxscores
            #     for i in range(len(stat_tables)):
            #         # split player and team boxscores
            #         player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)
        
            #     # team_stats database:
        
            #     # combine boxscore and advanced boxscore for each team
            #     away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
            #     team_boxes = pd.concat([away_team_box, home_team_box])
            #     team_boxes.reset_index(drop=True, inplace=True)
            #     # prepare numeric data
            #     team_boxes = change_dtypes(team_boxes, num_columns)
            #     # write to sql database
            #     team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)
        
            #     # player_stats database:
        
            #     # combine boxscore and advanced boxscore for each team
            #     away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
            #     player_boxes = pd.concat([away_player_box, home_player_box])
            #     player_boxes.reset_index(drop=True, inplace=True)
            #     # prepare numeric data
            #     player_boxes = change_dtypes(player_boxes, num_columns)
            #     # create team totals for PIE calculation
            #     totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
            #     # add PIE column to player boxscore
            #     player_boxes = create_PIE(player_boxes, totals)
            #     # write to sql database
            #     player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)
        
                
            #     button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')
        
            #     buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')
        
            #     del buttons[6]
            #     del buttons[3]
            #     del buttons[0]
        
            #     game_length = len(buttons)
        
            #     button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
            #     game_button_codes = button_codes[:game_length]
        
            #     for team_id in team_ids:
            #         for period in game_button_codes:
        
            #             id_string = f'box-{team_id}-{period}-basic'
            #             period_table = parser.find('table', attrs = {'id': id_string})
        
            #             player_box, team_box = create_boxscores(period_table, game_id=game_id)
        
            #             # team_stats database:
        
            #             team_box.rename(columns={'Starters': 'team'}, inplace=True)
            #             team_box['team'] = team_id
            #             team_box.insert(loc=1, column='period', value=period)
        
            #             team_box.reset_index(drop=True, inplace=True)
            #             team_box = change_dtypes(team_box, basic_stats)
        
            #             team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)
        
            #             # player_stats database:
        
            #             player_box.insert(loc=2, column='team', value=team_id)
            #             player_box.insert(loc=1, column='period', value=period)
        
            #             player_box.reset_index(drop=True, inplace=True)
            #             player_box = change_dtypes(player_box, basic_stats)
        
            #             player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)
        
            #     # increase gamecount to create next game_id
            #     season_gamecount += 1
        
            
            game_type = 'playoff'
            for i in range(len(playoff_game_urls)):
        
                # driver.quit()
                # time.sleep(rand)
                # driver = webdriver.Chrome(service=svc,options=options)
                # time.sleep(rand)
                driver.get(playoff_game_urls[i])
                time.sleep(rand)
                src = driver.page_source
                parser = BeautifulSoup(src, 'lxml')
        
                # game_info database:
        
                id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
                game_info = create_game_info(url=playoff_game_urls[i],
                                             season_id=season_id,
                                             season_gamecount=season_gamecount,
                                             game_type=game_type)
                # will use game_id with create_boxscores()
                game_id = game_info[0]
                team_info = create_team_info(id_table)
                # will use team_ids with merge_boxscores()
                team_ids = [team_info[0], team_info[2]]
        
                info_df = create_info_df(game_info=game_info,
                                         team_info=team_info,
                                         info_columns=info_columns)
                # write game info to sql database
                info_df.to_sql('game_info', con=conn, if_exists='append', index=False)
        
                # team/player databases:
        
                # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
                stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})
        
                player_box_list = [None, None, None, None]
                team_box_list = [None, None, None, None]
        
                # create team and player boxscores
                for i in range(len(stat_tables)):
                    # split player and team boxscores
                    player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)
        
                # team_stats database:
        
                # combine boxscore and advanced boxscore for each team
                away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
                team_boxes = pd.concat([away_team_box, home_team_box])
                team_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                team_boxes = change_dtypes(team_boxes, num_columns)
                # write to sql database
                team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)
        
                # player_stats database:
        
                # combine boxscore and advanced boxscore for each team
                away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
                player_boxes = pd.concat([away_player_box, home_player_box])
                player_boxes.reset_index(drop=True, inplace=True)
                # prepare numeric data
                player_boxes = change_dtypes(player_boxes, num_columns)
                # create team totals for PIE calculation
                totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
                # add PIE column to player boxscore
                player_boxes = create_PIE(player_boxes, totals)
                # write to sql database
                player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)
        
                # quarterly boxscores:
                button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')
        
                # button order: Game, Q1, Q2, H1, Q3, Q4, H2, (OT1, OT2, ...)
                buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')
        
                del buttons[6]
                del buttons[3]
                del buttons[0]
        
                game_length = len(buttons)
        
                button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
                game_button_codes = button_codes[:game_length]
        
                for team_id in team_ids:
                    for period in game_button_codes:
        
                        id_string = f'box-{team_id}-{period}-basic'
                        period_table = parser.find('table', attrs = {'id': id_string})
        
                        player_box, team_box = create_boxscores(period_table, game_id=game_id)
        
                        # team_stats database:
        
                        team_box.rename(columns={'Starters': 'team'}, inplace=True)
                        team_box['team'] = team_id
                        team_box.insert(loc=1, column='period', value=period)
        
                        team_box = change_dtypes(team_box, basic_stats)
        
                        team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)
        
                        # player_stats database:
        
                        player_box.insert(loc=2, column='team', value=team_id)
                        player_box.insert(loc=1, column='period', value=period)
        
                        player_box = change_dtypes(player_box, basic_stats)
        
                        player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)
        
                # increase gamecount to create next game_id
                season_gamecount += 1

In [ ]:
# driver.quit()
# conn.close()

In [3]:
conn = sqlite3.connect('data/temp/postcovid-current-season.sqlite')

In [6]:
query = 'SELECT * from team_stats_game'
game_info = pd.read_sql(query, con=conn)
# # account for team name change following the 2013-2014 season
# # game_info['away_team'].replace('CHA', 'CHO', inplace=True)
# # game_info['home_team'].replace('CHA', 'CHO', inplace=True)
game_info

,game_id,team,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,TS%,eFG%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
0,222410240001,LAL,240,41.0,90.0,0.456,10.0,29.0,0.345,15.0,20.0,0.750,13.0,31.0,44.0,23.0,5.0,4.0,11.0,18.0,107.0,None,0.541,0.511,0.322,0.222,28.3,77.5,51.2,56.1,5.2,7.0,10.0,100.0,112.3,124.8,None
1,222410240001,DEN,240,48.0,91.0,0.527,14.0,34.0,0.412,9.0,12.0,0.750,9.0,33.0,42.0,29.0,9.0,6.0,11.0,15.0,119.0,None,0.618,0.604,0.374,0.132,22.5,71.7,48.8,60.4,9.4,9.8,10.3,100.0,124.8,112.3,None
2,222410240002,PHO,240,42.0,95.0,0.442,11.0,33.0,0.333,13.0,17.0,0.765,17.0,43.0,60.0,23.0,5.0,7.0,19.0,22.0,108.0,None,0.527,0.500,0.347,0.179,35.4,70.5,55.0,54.8,4.9,12.1,15.6,100.0,106.2,102.3,None
3,222410240002,GSW,240,36.0,101.0,0.356,10.0,43.0,0.233,22.0,28.0,0.786,18.0,31.0,49.0,19.0,11.0,6.0,11.0,23.0,104.0,None,0.459,0.406,0.426,0.277,29.5,64.6,45.0,52.8,10.8,9.7,8.8,100.0,102.3,106.2,None
4,222410250003,HOU,240,32.0,79.0,0.405,12.0,34.0,0.353,10.0,16.0,0.625,7.0,24.0,31.0,19.0,6.0,3.0,18.0,24.0,86.0,None,0.500,0.481,0.430,0.203,14.9,58.5,35.2,59.4,6.4,5.7,17.3,100.0,91.8,123.8,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,222411300271,OKC,240,50.0,94.0,0.532,14.0,42.0,0.333,19.0,22.0,0.864,8.0,40.0,48.0,29.0,9.0,6.0,7.0,24.0,133.0,None,0.641,0.606,0.447,0.234,19.0,85.1,53.9,58.0,9.0,12.5,6.3,100.0,132.7,109.7,None
542,222411300272,ATL,240,49.0,96.0,0.510,14.0,33.0,0.424,25.0,29.0,0.862,8.0,29.0,37.0,24.0,14.0,2.0,10.0,25.0,137.0,None,0.630,0.583,0.344,0.302,18.2,72.5,44.0,49.0,13.0,4.2,8.4,100.0,127.1,125.2,None
543,222411300272,SAS,240,48.0,89.0,0.539,18.0,41.0,0.439,21.0,23.0,0.913,11.0,36.0,47.0,34.0,4.0,7.0,21.0,22.0,135.0,None,0.681,0.640,0.461,0.258,27.5,81.8,56.0,70.8,3.7,11.1,17.5,100.0,125.2,127.1,None
544,222411300273,LAC,240,42.0,95.0,0.442,12.0,32.0,0.375,18.0,24.0,0.750,15.0,28.0,43.0,26.0,5.0,1.0,6.0,21.0,114.0,None,0.540,0.505,0.337,0.253,28.8,73.7,47.8,61.9,5.4,2.3,5.4,100.0,123.8,130.4,None


## HERE HERE HERE

In [22]:
# gids = game_info['game_id'].apply(str)'
gids = [str(gid) for gid in game_info['game_id']]
gids = [gid.replace('2224', '2324') for gid in gids]
gids
# 

['232410240001',
 '232410240002',
 '232410250003',
 '232410250004',
 '232410250005',
 '232410250006',
 '232410250007',
 '232410250008',
 '232410250009',
 '232410250010',
 '232410250011',
 '232410250012',
 '232410250013',
 '232410250014',
 '232410260015',
 '232410260016',
 '232410270017',
 '232410270018',
 '232410270019',
 '232410270020',
 '232410270021',
 '232410270022',
 '232410270023',
 '232410270024',
 '232410270025',
 '232410270026',
 '232410270027',
 '232410280028',
 '232410280029',
 '232410280030',
 '232410280031',
 '232410280032',
 '232410280033',
 '232410280034',
 '232410290035',
 '232410290036',
 '232410290037',
 '232410290038',
 '232410290039',
 '232410290040',
 '232410300041',
 '232410300042',
 '232410300043',
 '232410300044',
 '232410300045',
 '232410300046',
 '232410300047',
 '232410300048',
 '232410300049',
 '232410300050',
 '232410300051',
 '232410310052',
 '232410310053',
 '232410310054',
 '232411010055',
 '232411010056',
 '232411010057',
 '232411010058',
 '232411010059

In [ ]:
query = '''DELETE from game_info
WHERE game_id LIKE "2122%"
'''

cursor = conn.cursor()
cursor.execute(query)
conn.commit()

In [ ]:
query = 'SELECT * from team_stats_game'
team_stats_quarterly = pd.read_sql(query, con=conn)
# # account for team name change following the 2013-2014 season
# # game_info['away_team'].replace('CHA', 'CHO', inplace=True)
# # game_info['home_team'].replace('CHA', 'CHO', inplace=True)
team_stats_quarterly

In [ ]:
gids = [int.from_bytes(x, byteorder='little') for x in team_stats_quarterly['game_id']]
gids

In [ ]:
team_stats_quarterly['game_id'] = gids

In [ ]:
team_stats_quarterly

In [ ]:
query = '''DELETE from team_stats_quarterly

'''

cursor = conn.cursor()
cursor.execute(query)
conn.commit()

In [ ]:
team_stats_quarterly.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

In [ ]:
query = '''DELETE from team_stats_quarterly
WHERE game_id LIKE "2122%"
'''

cursor = conn.cursor()
cursor.execute(query)
conn.commit()

In [ ]:
query = 'SELECT * from player_stats_game'
team_stats_quarterly = pd.read_sql(query, con=conn)
# # account for team name change following the 2013-2014 season
# # game_info['away_team'].replace('CHA', 'CHO', inplace=True)
# # game_info['home_team'].replace('CHA', 'CHO', inplace=True)
team_stats_quarterly

In [ ]:
conn.close()

In [ ]:
conn = sqlite3.connect('data/temp/nba-db-current.sqlite')